# DS4DS Homework Exercise Sheet 11

**General Instructions:**

- Collaborations between students during problem-solving phase on a discussion basis is OK
- However: individual code programming and submissions per student are required
- Code sharing is strictly prohibited
- We will run checks for shared code, general plagiarism and AI-generated solutions
- Any fraud attempt will lead to an auto fail of the entire course
- Do not use any additional packages except for those provided in the task templates
- Please use Julia Version 1.10.x to ensure compatibility
- Please only write between the `#--- YOUR CODE STARTS HERE ---#` and `#--- YOUR CODE ENDS HERE ---#` comments
- Please do not delete, add any cells or overwrite cells other than the solution cells (**Tip:** If you use a jupyerhub IDE, you should not be able to add or delete cells and write in the non-solution cells by default)

In [ ]:
using Printf
using DelimitedFiles, Plots, GZip, StatsBase, LaTeXStrings, Serialization
using OrdinaryDiffEq, Lux, Random, ComponentArrays
using DataInterpolations, SciMLSensitivity, Optimization, OptimizationOptimJL, LineSearches, Zygote, OptimizationFlux, Flux

## Task 1: NODE model for a SISO heating system - (6 points)

In this task, we consider a simple single-input-single-output (SISO) heating system. The input of the system $u(t)$ is the voltage that drives a  halogen lamp which is suspended several inches above a thin steel plate. The output of the system $y(t)$ is a thermocouple measurement taken from the back of the plate ([source](https://ftp.esat.kuleuven.be/pub/SISTA/data/thermic/) and [description](https://ftp.esat.kuleuven.be/pub/SISTA/data/thermic/heating_system.txt) of the dataset).

Our goal within this task will be to identify the dynamics of the underlying process. We make the assumption that the temperature measurment holds sufficient information to characterize the process, i.e., that $\hat{x}(t) = \hat{y}(t)$ applies. We will build a model

$$
\begin{align}
    \frac{\mathrm{d}}{\mathrm{d}t} \hat{x}(t) &= \mathcal{M}_{\mathbf{w}} (\hat{x}(t), u(t)) \\
    \hat{y}(t) & = \hat{x} (t)
\end{align}
$$

that estimates the change of temperature. To this end, we will make use of neural ordinary differential equations (NODEs).

The basic idea of NODEs is to use an artifical neural network (ANN) to model the state transition equation $\frac{\mathrm{d}}{\mathrm{d}t} \hat{x}(t) = \mathcal{M}_{\mathbf{w}} (\hat{x}(t), u(t))$ that is parameterized by $\mathbf{w}$. The resulting ODE is then solved by standard ODE solvers to yield an estimate for the state trajectory. This estimated state trajectory is then compared to the target trajectory to optimize the parameters in the ANN and improve the approximation.

#### Function for dataset loading:

In [ ]:
function loaddata_heating(; normalize=true)
    fh = GZip.open("heating_system.dat.gz")
    data = readdlm(fh)

    data_freq = 4
    
    data = data[1:data_freq:end, :]
    
    t = data[:, 1] / 16  # time in seconds
    u = data[:, 2]  # input: voltage driving the lamp in volt
    y = data[:, 3]  # output: temperature measurement in °C

    
    dt = 2 * data_freq / 16  # time between samples in seconds (thermic process -> high inertia)
    
    # convert to Float32 for performance
    t = Float32.(t)
    u = Float32.(u) 
    y = Float32.(y)
    
    if normalize  # optional standardization of data
        dty = fit(ZScoreTransform, y)
        dtu = fit(ZScoreTransform, u)
    
        y = StatsBase.transform(dty, y)
        u = StatsBase.transform(dtu, u);
    end
    
    tspan = (t[1], t[end])
    
    return u, y, dt, tspan, t
end

In [ ]:
u, y, dt, tspan, tsteps = loaddata_heating(normalize = false)
p = plot(tsteps, u, label=L"u", title="Simple heating system", xlabel=L"$t$ in seconds", ylabel=L"$u$ in V / $y$ in °C",
     legend=:topleft, lw=2, palette = :Set2_5)
plot!(p, tsteps, y, label=L"y", lw =2)

#### **a)** - (0.5 point)
Split your dataset into a training and evaluation dataset. The training dataset will be used to tune the parameters of your model and the evaluation data will be used to evaluate the generalization performance of your model.
Use $u(t >= 37.5s)$ and $y(t >= 37.5s)$ for the training and $u(t < 37.5s)$ and $y(t < 37.5s)$ for the evaluation datasets. Additionally, provide the corresponding timesteps for the training and eval datasets, i.e., provide
```tsteps_training``` which contains all timesteps from the dataset with $t >= 37.5s$ and ```tsteps_eval``` which contains all timesteps with $t < 37.5s$.

Implement the function below to split the dataset.

In [ ]:
u, y, dt, tspan, tsteps = loaddata_heating();

function split_dataset(u, y, tsteps)
    """
    Creates a training and evaluation split
    
    Args:
        u: System input data
        y: System output data
        tsteps: time steps of the data trajectories

    Returns:
        u_training: u(t >= 37.5s)
        y_training: y(t >= 37.5s)
        u_eval: u(t < 37.5s)
        y_eval: y(t < 37.5s)
        tsteps_training: time steps for the training data
        tsteps_eval: time steps for the eval data
    """
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#

    return u_training, y_training, u_eval, y_eval, tsteps_training, tsteps_eval
end
    
u_training, y_training, u_eval, y_eval, tsteps_training, tsteps_eval = split_dataset(u, y, tsteps);

In [ ]:
@assert isa(split_dataset, Function)
@assert vcat(u_eval, u_training) == u  "Full dataset needs to be the concatenation of training and eval sets"
@assert vcat(y_eval, y_training) == y  "Full dataset needs to be the concatenation of training and eval sets"


In [ ]:
# Please leave this cell as it is

#### **b)** - (1 point)
Implement a function for the creation of a fully-connected feed-forward neural network in dependence of the layer sizes and the activation functions. Use ```Lux.jl``` to implement it in the function template given below.

In [ ]:
function create_ann(layer_sizes, hidden_layer_activation_function, output_activation)
    """
    Creates a neural network with the given parameters

    Args:
        layer_sizes: List of layer sizes (e.g. [5, 32, 32, 2])
        hidden_layer_activation_function: Activation function for all hidden layers (nomenclature of Lux.jl)
        output_activation: Activation function of the output layer (nomenclature of Lux.jl)

    Returns:
        ann: Resulting fully-connected neural network
    """
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#

    return ann
end

# the sizes you choose here have no influence on the grading of this subtask
layer_sizes = nothing
hidden_layer_activation_function = nothing
output_activation = nothing

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

In [ ]:
ann = create_ann(layer_sizes, hidden_layer_activation_function, output_activation)
@assert isa(ann, Lux.Chain)


#### **c)** - (1 point)

In this subtask, we want to define the NODE of the form

$$
\begin{align}
    \frac{\mathrm{d}}{\mathrm{d}t} \hat{x}(t) &= \mathcal{M}_{\mathbf{w}} (\hat{x}(t), u(t)).
\end{align}
$$

To this end, we need to define the ODE in the function ```dxdt(x, w, t)``` and initialize the ```training_problem```.
Use ```create_ann``` from subtask **b)** to create a model for the ODE. If you were unable to properly implement it in **b)**, you are also allowed to create a model manually. However, you are restricted to fully-connected feed-forward neural networks. Use the variable ```w``` for the model parameters $\mathbf{w}$.

The layer sizes, activation functions and initialization are set here to help you with the following subtask.
If you want to design a different network, you are free to do so. Note, however that you might face a more complex optimization in that case.

In [ ]:
# theoretically you would need to interpolate separately for training and evaluation, but we will look past this here
# as the time is already dividing between the two
interp_input = AkimaInterpolation(u, tsteps);

ann = create_ann([2, 8, 4, 1], tanh, x -> x)

# initialize parameter and state variables

# using this seed for initialization the intial state of the model is quite good for the task,
# otherwise the optimization is too complex for the given ressources..
seed = 2 

w, st = Lux.setup(MersenneTwister(seed), ann);
w = ComponentArray(w);

dxdt(x, w, t) = nothing  # ODE function
training_prob = nothing  # ODE training problem

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

In [ ]:
@assert isa(dxdt, Function)
@assert isa(training_prob, ODEProblem)


#### **d)** - (0.5 points)
Next, we want to implement the forward pass of the NODE.

Implement the function below that takes the parameters of the model $\mathbf{w}$, an ODE problem, the initial state $x_0$ and the timesteps and returns the state trajectory $\hat{\mathbf{x}}_n$.
Use the Tsit5 ODE solver, but non-adaptive.

In [ ]:
function predict_NODE(w, problem, x0, tsteps, dt)
    """
    NODE forward pass

    Args:
        w: Parameters of the model as a ComponentArray
        problem: ODE problem
        x0: Initial state of the trajectory
        tsteps: Time steps of the trajectory
        dt: The time between two samples in the trajectory

    Returns:
        Resulting state trajectory
    
    """
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#
end;

In [ ]:
@assert isa(predict_NODE, Function)


#### **e)** 
Implement the MSE loss function to compare model predictions and true trajectory. Optimize the parameters $\mathbf{w}$ of your model given in subtask c) using the given optimizer.
Design an adequate optimization problem for the given template.

**Note:** The code itself will generally not be evaluated in this subtask, but only the performance on the evaluation set. We will inspect your optimization routine more closely if it looks suspicious.

In [ ]:
#your final model parameters
result = nothing

optimizer = OptimizationFlux.Adam(1e-3)
maxiters = 2_000

function callback(state, l)
	state.iter % 100 == 0 && @printf "Iteration: %5d, Training loss: %.6e\n" state.iter l

    # extend the callback if you like, the parameters can be accessed as state.u
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#
	return false
end

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

result = Optimization.solve(
    optprob,
    optimizer,
    maxiters=maxiters,
    callback=callback
)

# save your model parameters
if @isdefined result
    #Save your model as file
    @assert isa(result, SciMLBase.OptimizationSolution)
    serialize("model_task_1",result)
end

After you have trained your model, save the parameters using the code above into the file `model_task_1`. 

**Then comment out your optimization code in the cell above and remove your saving code so that both do not run in the autograding!**

In [ ]:
# load your model parameters

student_model = deserialize("model_task_1")
@assert isa(student_model, SciMLBase.OptimizationSolution)

student_w = student_model.u  # extract parameters

#### **f)** - (3 points)
Evaluate the performance of your model on the evaluation dataset. To do so, define the evaluation problem.

To gain the points for this subtask, you have to produce a model that reaches an MSE between the prediction and the eval trajectory lower than the threshold given below.
The previously given architecture and optimization parameters should help you reach this prediction accuracy.

In [ ]:
x0 = nothing
evaluation_problem = nothing

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

In [ ]:
@assert isa(x0,Number)
@assert isa(evaluation_problem, ODEProblem)

prediction_eval = predict_NODE(student_w, evaluation_problem, [x0] , tsteps_eval, dt)[1,:]

@assert length(prediction_eval) == length(tsteps_eval)

prediction_error_eval = Flux.mse(prediction_eval, y_eval)
println("Given prediction error: ", prediction_error_eval)

@assert prediction_error_eval <= 0.3

In [ ]:
prediction_training = predict_NODE(student_w, training_prob, [y_training[1]] , tsteps_training, dt)[1,:]

p = plot(tsteps_training, y_training, label=L"y", lw =2, title="Training set performance")
plot!(p, tsteps_training, u_training, label=L"u", lw =2)
plot!(p, tsteps_training, prediction_training, label=L"\hat{y}", lw =2)

In [ ]:
eval_plot = scatter(tsteps_eval, y_eval, label=L"$y(t)$", title="Eval performance", xlabel=L"$t$ in seconds", ylabel=L"$y$ in °C", legend=:bottomleft, lw=2)
plot!(eval_plot, tsteps_eval, prediction_eval, label=L"$\hat{y}(t)$", lw =2)

## Task 2: NODE model for Lake Erie (MIMO) - (4 points)

In this task, we once again model an underlying process using NODEs. However, the system at hand now takes multiple inputs $\mathbf{u}(t)$ and yields multiple outputs $\mathbf{y}(t)$. This means we need to model a multiple-input-multiple-output (MIMO) system.

This results in the dynamics equations 

$$
\begin{align}
    \frac{\mathrm{d}}{\mathrm{d}t} \hat{\mathbf{x}}(t) &= \mathcal{M}_{\mathbf{w}} (\hat{\mathbf{x}}(t), \mathbf{u}(t)) \\
    \hat{\mathbf{y}}(t) & = \hat{\mathbf{x}} (t).
\end{align}
$$

The data ([source](https://ftp.esat.kuleuven.be/pub/SISTA/data/environmental/) and [description](https://ftp.esat.kuleuven.be/pub/SISTA/data/environmental/erie.txt) of the dataset) is the result of a simulation of the western basin of Lake Erie. The inputs $\mathbf{u}(t)$ are the water temperature, water conductivity, water alkalinity, the NO3 content and the total hardness of the water. The outputs $\mathbf{y}(t)$ are the amount of dissolved oxigen and the algae content. We are interested in the dynamic relationship between inputs and outputs described by the model above.

#### a) - (1 point)

Below you are shown the raw output data:

![lake_erie-not_normalized](./LakeErie-NotNormalized.svg)

You can see that the two outputs are scaled very differently. As a result, we need to normalize them separately.

Implement the function below to normalize each output separately. Use `StatsBase.fit(ZScoreTransform, ...)` and `StatsBase.transform(...)` for the normalization.

In [ ]:
function normalize_columnwise(data)

    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#
    
    return normalized_data
end

function loaddata_erie(; normalize= true)
    fh = GZip.open("erie.dat.gz")
    data = readdlm(fh)
    
    t = data[:, 1]  # time in months
    u = data[:, 2:6]
    y = data[:, 22:23]
    
    dt = 1  # time between samples in months
    
    # convert to Float32 for performance
    t = Float32.(t)
    u = Float32.(u) 
    y = Float32.(y)
    
    if normalize  # optional standardization of data
        y = normalize_columnwise(y)
        u = normalize_columnwise(u)
    end
    
    tspan = (t[1], t[end])
    
    return u, y, dt, tspan, t
end

Code for plot below when the data is normalized properly:
```
p = plot(tsteps, y, label=L"y", title="Lake Erie", xlabel=L"$t$ in months", ylabel=L"y", legend=:topleft, background_color="#000000",lw=2, palette = :Set2_5)
```

![lake_erie](./LakeErie.svg)

In [ ]:
isa(normalize_columnwise, Function)


In [ ]:
# please leave this cell as it is


#### Preparations:

In [ ]:
u, y, dt, tspan, tsteps = loaddata_erie(normalize=true)
print(size(u))

# interpolate inputs for all channels separately
interp_input = [AkimaInterpolation(u[:, i], tsteps) for i in 1:size(u, 2)]
function interpolate_input(t)
    return [f(t_) for (f, t_) in zip(interp_input, ones(size(interp_input, 1))* t)]
end

In [ ]:
belonging_to_training = tsteps .<= 40
belonging_to_eval = tsteps .> 40

u_training = u[belonging_to_training, :]
y_training = y[belonging_to_training, :]
tsteps_training = tsteps[belonging_to_training, :]

u_eval = u[belonging_to_eval, :]
y_eval = y[belonging_to_eval, :]
tsteps_eval = tsteps[belonging_to_eval, :];

tsteps_training = vec(tsteps_training)
tsteps_eval = vec(tsteps_eval);

#### **b)**

Build and train a NODE model using the training dataset. Design a feed-forward fully-connected neural network as the model.

In [ ]:
result_2 = nothing

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#


if @isdefined result_erie
    serialize("model_task_2", result_erie)
end

After you have trained your model, save the parameters using the code above into the file `model_task_2`. 

**Then comment out your optimization code in the cell above and remove your saving code so that both do not run in the autograding!**

**Ensure that everything that is necessary for the evaluation is still defined (Model definition etc.), i.e. that the evaluation can still run at submission!**

#### **c)** - (3 points)

Evaluate the performance of your model on the evaluation dataset. To do so, define the evaluation problem.

To gain the points for this subtask, you have to produce a model that reaches an MSE between the prediction and the eval trajectory lower than the threshold given below. Design a model architecture and optimization routine in the previous subtasks that reaches this level of accuracy.

In [ ]:
result_erie = deserialize("model_task_2")
student_w = result_erie.u  # extract parameters

In [ ]:
x0_eval = nothing
evaluation_prob = nothing

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

In [ ]:
#public test
@assert isa(x0_eval, Array)
@assert isa(evaluation_prob, ODEProblem)

prediction_eval = transpose(predict_NODE(student_w, evaluation_prob, x0_eval, tsteps_eval, dt))


@assert length(prediction_eval) == length(y_eval)

prediction_error_eval = Flux.mse(prediction_eval, y_eval)
println("Given prediction error: ", prediction_error_eval)

@assert prediction_error_eval <= 0.5

In [ ]:
prediction_training = transpose(predict_NODE(student_w, training_problem, y_training[1, :] , tsteps_training, dt))

p = plot(tsteps_training, y_training, label=L"y", title="Lake Erie", xlabel=L"$t$ in months", ylabel=L"y", legend=:topleft,lw=2, palette = :Set2_5)
# plot!(p, tsteps_training, u_training, label=L"u", lw =2)
plot!(p, tsteps_training, prediction_training, label=L"\hat{y}", lw =2)

In [ ]:
prediction_eval, prediction_error_eval = evaluate_result(student_w, y_eval, tsteps_eval)
eval_plot = Plots.plot(tsteps_eval, y_eval, label=L"$y(t)$", title="Simple heating system", xlabel=L"$t$ in seconds", ylabel=L"$y$ in °C", legend=:bottomleft,lw=2, palette = :Set2_5)
Plots.plot!(eval_plot, tsteps_eval, prediction_eval, label=L"$\hat{y}(t)$", lw =2)
display(eval_plot)